<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/ex13notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2022 ex13notebookB

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo13.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2022

In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の DCT と FFT の関数
from scipy.fft import dct, fft, ifft, fftfreq

# 科学技術計算のライブラリ SciPy の中の WAVE ファイルを扱うモジュール
from scipy.io.wavfile import read, write

----
## 演習問題 大津市の気温の DCT と DFT
----



---
### データの準備

大津市の2004年度の気温（日ごとの平均気温）のデータを読み込んで `temp2004` という変数に代入します．この年はうるう年ですので，値が $366$ 個あります．

In [ ]:
temp2004 = np.loadtxt('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/temp2004.txt')
N = len(temp2004)
temp2004

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(temp2004, '-', color='blue', label='temp2004')
ax.axhline(0, color='gray')
ax.legend()
plt.show()

以下，この気温のデータを表す $N$ 次元ベクトルを $\mathbf{x}$ と表記することにします．
記号をちゃんと定義せずに使っているところがあります．notebookA を参照して補ってね．

---
### DCT

上記の気温のデータをDCTしてみましょう．

次のセルを実行すると，$N$ 次元のDCTの基底のうち最初の4つ $\mathbf{u}_0, \mathbf{u}_1, \mathbf{u}_2, \mathbf{u}_3$ をグラフに描きます．曲線に見えますが，実際にはそれぞれ $N$ 個の点がプロットされています．

In [ ]:
# DCT の基底を求める
N = len(temp2004)
U_dct = dct(np.eye(N), norm='ortho')

# 0 番から 3 番の基底ベクトルを可視化
fig, ax = plt.subplots(figsize=(8, 6))
for k in range(4):
    ax.plot(U_dct[:, k], '.', label=f'u{k}')
ax.axhline(0, color='gray')
ax.set_ylim(-0.1, 0.1)
ax.legend()
plt.show()

#### 問題1

次のことを考えて／調べてメモしておこう：

(1) ここでは $N$ はいくつ？ 基底は，何次元のベクトル何本から成る？

(2) 描かれている4つの基底ベクトルのうち，366日周期の波はどれ？



次のセルを実行すると，`temp2004` にDCTを適用します．`c_dct` という配列に展開係数の値 $c_k$ ($k = 0, 1, \ldots, N-1$)が格納されます．

In [ ]:
# temp2004 の DCT
c_dct = dct(temp2004, norm='ortho')
print(c_dct)

次のセルを実行すると，`c_dct` の値を可視化します．横軸が $k$ で縦軸が $c_k$ です．

In [ ]:
# 展開係数の可視化
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
ax[0].stem(c_dct, markerfmt=' ', use_line_collection=True)
ax[0].set_ylim(-330, 330)
ax[1].stem(c_dct, markerfmt='o', use_line_collection=True)
ax[1].set_xlim(-0.5,10.5)
ax[1].set_ylim(-330, 330)
plt.show()

次のセルを実行すると，$0$ 以上 $N-1$ 未満の整数 $H$ に対して

$$
\begin{aligned}
\mathbf{z}_\textrm{low} &= c_0\mathbf{u}_0 + c_1\mathbf{u}_1 + \cdots + c_{H}\mathbf{u}_{H}\\
\mathbf{z}_\textrm{high} &= c_{H+1}\mathbf{u}_{H+1} +  \cdots + c_{N-1}\mathbf{u}_{N-1}
\end{aligned}
$$

を計算し，$\mathbf{x}$ と重ねて表示します．


In [ ]:
H = 0 #@param {type:"number", min:0, max:366}
assert 0 <= H and H < N-1

fig, ax = plt.subplots(1, 2, figsize=(16, 6))

z_low = c_dct[:H+1] @ U_dct[:, :H+1].T
ax[0].plot(temp2004, '-', label='temp2004')
ax[0].plot(z_low, '-', color='red', label='z_low', linewidth=2)
ax[0].axhline(0, color='gray')
ax[0].legend()

z_high = c_dct[H+1:] @ U_dct[:, H+1:].T
ax[1].plot(temp2004, '-', label='temp2004')
ax[1].plot(z_high, '-', color='red', label='z_high', linewidth=2)
ax[1].axhline(0, color='gray')
ax[1].legend()

plt.show()

$\mathbf{z}_\textrm{low}$ の方は，元のベクトルから長い周期の（周波数の低い）波の成分のみを取り出して表したものとなっており，$\mathbf{z}_\textrm{high}$ の方は逆に，短い周期の（周波数の高い）波の成分のみを取り出して表したものとなっています．

#### 問題2

次のことをやろう／考えよう．

(1) $H = 0, 1, 2, 3$ で実行して結果を観察しよう．もっと大きな $H$ でもやってみよう．

(2) $k = 104$ のとき $\frac{2N}{k} =7.04$ なので，$\mathbf{u}_{104}$ はおよそ1週間周期の変動の成分と言えます．$H=104$ として実行しよう．$\{\mathbf{u}_0, \ldots, \mathbf{u}_{104}\}$ と $\{\mathbf{u}_{105}, \ldots, \mathbf{u}_{366}\}$ のうち，1週間より短い周期での変動を表す成分はどっち？


---
### DFT

上記の気温のデータをDFTしてみましょう．

次のセルを実行すると，$N$ 次元のDFTの基底のうち最初の4つ $\mathbf{u}_0, \mathbf{u}_1, \mathbf{u}_2, \mathbf{u}_3$ をグラフに描きます．曲線に見えますが，実際にはそれぞれ $N$ 個の点がプロットされています．

In [ ]:
# DFT の基底を求める
N = len(temp2004)
U_dft = np.conj(fft(np.eye(N)))

# 0 番から 3 番の基底ベクトルを可視化
fig, ax = plt.subplots(2, 4, figsize=(16, 6))
xp = np.linspace(0, N-1, num=200)
for k in range(4):
    # 実部
    re = U_dft[:, k].real # 実部
    ax[0][k].plot(re, '-', color='blue', label=f'Re u{k}')
    # 虚部
    im = U_dft[:, k].imag # 虚部
    ax[1][k].plot(im, '-', color='red', label=f'Im u{k}')
    for i in range(2):
        ax[i][k].set_ylim(-1.4, 1.4)
        ax[i][k].axhline(0, color='gray')
        ax[i][k].legend()
plt.show()

#### 問題3

次のことを考えて／調べてメモしておこう：

(1) ここでは $N$ はいくつ？ 基底は，何次元のベクトル何本から成る？

(2) 描かれている4つの基底ベクトルのうち，366日周期の波はどれ？



次のセルを実行すると，`temp2004` にDFTを適用します．`c_dft` という配列に展開係数の値 $c_k$ ($k = 0, 1, \ldots, N-1$)が格納されます．

In [ ]:
# temp2004 の DFT
c_dft = fft(temp2004)
print(c_dft)

次のセルを実行すると，`c_dft` の値を可視化します．

In [ ]:
# 絶対値と偏角
amplitude = np.abs(c_dft) # 絶対値
phase = np.angle(c_dft) # 偏角
#print(amplitude)
#print(phase)

# フーリエ係数の可視化（振幅と位相）
fig = plt.figure(figsize=(12, 12))

# 振幅スペクトル
ax = fig.add_subplot(2, 2, 1)
ax.stem(amplitude, markerfmt=' ', use_line_collection=True, label='|Ck|')
ax.legend()

# 振幅スペクトル（縦軸は常用対数をとった値）
ax = fig.add_subplot(2, 2, 2)
ax.stem(np.log10(amplitude+1), markerfmt=' ', use_line_collection=True, label='|Ck|')
ax.legend()

# 振幅スペクトル（一部の範囲，縦軸は常用対数をとった値）
ax = fig.add_subplot(2, 2, 3)
ax.stem(np.log10(amplitude[:21]+1), markerfmt=' ', use_line_collection=True, label='|Ck|')
ax.set_xlim(-1, 22)
ax.legend()

# 位相スペクトル
ax = fig.add_subplot(2, 2, 4)
ax.stem(phase, markerfmt=' ', use_line_collection=True, label='arg Ck')
ax.legend()

plt.show()

左上は普通の振幅スペクトルです．横軸が $k$ で縦軸が $|c_k|$ です．右上は，縦軸を $\log_{10}|c_k|$ としたものです．値の変化の範囲が大きいときは，スペクトルをこのように対数プロットすることもあります．左下は，右上のものの一部を拡大して表示したものです．右下は，位相スペクトルです．縦軸が $\arg c_k$ で，値の範囲は $[-\pi, \pi]$ です．

次のセルを実行すると，$0$ 以上 $\frac{N}{2}$ 未満の整数 $H$ に対して

$$
\begin{aligned}
\mathbf{z} &= c_0\mathbf{u}_0 + c_1\mathbf{u}_1 + \cdots + c_{H}\mathbf{u}_{H}\\
&+ c_{N-H}\mathbf{u}_{N-H} + c_{N-H+1}\mathbf{u}_{N-H+1} \cdots + c_{N-1}\mathbf{u}_{N-1}
\end{aligned}
$$

を計算し，$\mathbf{x}$ と重ねて表示します．

In [ ]:
H = 0 #@param {type:"number"}
assert 0 <= H and H <= N//2

c_truncated = np.copy(c_dft)
c_truncated[H+1:N-H] = 0
z = ifft(c_truncated).real

fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(temp2004, '-', label='temp2004')
ax.plot(z, '-', color='red', label='z', linewidth=2)
ax.axhline(0, color='gray')
ax.legend()

plt.show()

#### 問題4

次のことをやろう／考えよう．

(1) 振幅スペクトルを見ると，定数成分以外で最も振幅が大きいのは，$k$ がいくつのもの？その周期はいくつ？

(2) $H = 0, 1, 2, 3, 5, 10$ で実行して結果を観察しよう．もっと大きな $H$ でもやってみよう．

(3) $H = 183$ とするとどうなる?



----
## 演習問題 - 音のデータの DFT
----

音のデータを分析してみましょう．

---
### 正弦波

データを準備します．コードのコメントにある「標本化周波数」や「量子化」といった話は次回解説予定です．今は気にしなくてokです．

In [ ]:
framerate = 8000       # 標本化周波数
vmax = 2**(15-2)       # 16bit量子化で最大振幅の 1/4
duration = 2*framerate # 2[s]

t = np.arange(duration)

freq1 = [200, 400, 600]
freq2 = [1200, 400, 800]

# signal1
signal1 = vmax * np.sin((2*np.pi*t/(framerate/freq1[0]))) \
 + vmax/2 * np.sin((2*np.pi*t/(framerate/freq1[1]))) \
 + vmax/4 * np.sin((2*np.pi*t/(framerate/freq1[2]))) \
 + vmax/2 * np.random.randn(len(t)) # ランダムノイズ

# signal2
signal2 = vmax * np.sin((2*np.pi*t/(framerate/freq2[0]))) \
 + vmax/2 * np.sin((2*np.pi*t/(framerate/freq2[1]))) \
 + vmax/4 * np.sin((2*np.pi*t/(framerate/freq2[2]))) \
 + vmax/2 * np.random.randn(len(t)) # ランダムノイズ

# グラフに描く
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(signal1, '-o', label='signal1')
ax.plot(signal2, '-o', label='signal2')
ax.axhline(0, color='gray')
ax.set_xlim(0, 100)
ax.legend()
plt.show()

`signal1` と `signal2` という2つの音のデータを作りました．どちらも，周波数の異なる3つの正弦波を足し合わせたものにノイズを乗せてあります．上のグラフからそれぞれがどんな音か想像する...のは難しいので，WAVE 形式のファイルとして出力して手元のPCにダウンロードし，音を鳴らしてみましょう．

1. 次のコードセルに記されたコメントに従ってコードを修正して実行
1. 手元に `signal1.wav` と `signal2.wav` というファイルがダウンロードされる．初めて実行すると，「複数ファイルの一括ダウンロードの許可」を求めるポップアップウィンドウが現れるかもしれません．その場合，許可するとダウンロードが始まります．
1. 手元のPCでダウンロードされたファイルを再生．WAVE 形式のファイルは，Windows でも macOS でも，OS標準の音楽プレイヤーで再生できるはずです．

In [ ]:
from google.colab import files

if 1 == 0: # ← の 0 を 1 に修正
    write('signal1.wav', framerate, signal1.astype('int16'))
    write('signal2.wav', framerate, signal2.astype('int16'))
    files.download('signal1.wav')
    files.download('signal2.wav')

次のセルを実行すると，`signal1` と `signal2` に DFT を適用してそれぞれのフーリエ係数を求め，振幅スペクトルを描きます．

In [ ]:
N = len(signal1)
freq = fftfreq(N, d=1/framerate)

# signal1 の振幅スペクトル
c_dft = fft(signal1)
amp1 = np.abs(c_dft)/N # 絶対値

# signal2 の振幅スペクトル
c_dft = fft(signal2)
amp2 = np.abs(c_dft)/N # 絶対値

fig, ax = plt.subplots(2, 3, figsize=(16, 8))

ax[0, 0].stem(amp1, markerfmt=' ', use_line_collection=True, label='signal1 |Ck|')
ax[0, 0].set_xlabel('k')
ax[0, 0].legend()

ax[0, 1].stem(amp1, markerfmt=' ', use_line_collection=True, label='signal1 |Ck|')
ax[0, 1].set_xlim(0, 3000)
ax[0, 1].set_xticks(np.arange(0, 3000, 400))
ax[0, 1].set_xlabel('k')
ax[0, 1].legend()

ax[0, 2].stem(freq, amp1, markerfmt=' ', use_line_collection=True, label='signal1 |Ck|')
ax[0, 2].set_xlim(0, 1500)
ax[0, 2].set_xticks(np.arange(0, 1500, 200))
ax[0, 2].set_xlabel('Hz')
ax[0, 2].legend()

ax[1, 0].stem(amp2, markerfmt=' ', use_line_collection=True, label='signal2 |Ck|')
ax[1, 0].set_xlabel('k')
ax[1, 0].legend()

ax[1, 1].stem(amp2, markerfmt=' ', use_line_collection=True, label='signal2 |Ck|')
ax[1, 1].set_xlim(0, 3000)
ax[1, 1].set_xticks(np.arange(0, 3000, 400))
ax[1, 1].set_xlabel('k')
ax[1, 1].legend()

ax[1, 2].stem(freq, amp2, markerfmt=' ', use_line_collection=True, label='signal2 |Ck|')
ax[1, 2].set_xlim(0, 1500)
ax[1, 2].set_xticks(np.arange(0, 1500, 200))
ax[1, 2].set_xlabel('Hz')
ax[1, 2].legend()

plt.show()

上段が `signal1` の振幅スペクトル，下段が `signal2` の振幅スペクトルです．
左は，横軸 $k$，縦軸 $|c_k|$ で横軸の範囲を $[0, N-1]$ として描いたもの，真ん中は，横軸の範囲を狭めたものです．
右の図では，横軸の範囲は真ん中と同じですが，横軸の単位を周波数 [Hz] として描いています．
$k$ と周波数との対応については次回説明しますので，ここではそういうもんだと思って眺めればokです．

`signal1` と `signal2` のどちらの音も，周波数の異なる3つの正弦波を足し合わせたものにノイズを乗せて作ってあります．元のデータの波形を眺めたり音を聞いたりしただけではどんな成分が含まれているのか分かりづらいですが，振幅スペクトルを見ると一目瞭然となっています．

#### 問題5

次のことをやろう／考えよう．

(1) `signal1` と `signal2` は，どちらの方が高い音に聞こえますか？

(2) それぞれの音に含まれる成分のうち，主要な（振幅が大きい）成分は，周波数が何Hzのものですか？ それぞれの音で3つずつ答えなさい．

(3) 振幅スペクトルによると，`signal1` と `signal2` ではどちらの方が高い周波数の成分を含んでいますか？ （(2)で求めた主要な成分に注目して考えよう）




---
### もふもふから出る音

別の音のデータで実験しましょう．

ネットから WAVE 形式のファイルを入手します．まずは，WAVE形式のファイルを読み込む関数を定義します．

In [ ]:
# WAVE ファイルを読み込む関数
#
def readWAVE(filename):
    
    framerate, data = read(filename)

    # チャンネル数とフレーム数（データ点の数）を求める
    if data.ndim == 1:
        nchannels = 1
    else:
        nchannels = data.shape[1]
    nframes = data.shape[0]
    
    print('filename = ', filename)
    print('nchannels = ', nchannels)       # チャンネル数
    print('framerate = ', framerate)       # 標本化周波数
    print('nframes = ', nframes)             # フレーム数
    print('duration = {0:.2f}[sec]'.format(nframes / framerate))   # 長さ（秒単位）
    print('dtype = ', data.dtype)            # データ型（量子化ビット数に対応）

    assert data.dtype == 'uint8' or data.dtype == 'int16' or data.dtype == 'int32' or data.dtype == 'float32'
    
    # 最初の10秒分だけ取り出す（元がそれより短ければそのまま）
    nframesNew = min(framerate * 10, nframes)   
    if nchannels == 1:
        dataNew = data[:nframesNew]
    else:
        # 多チャンネル信号なら0番目と1番目の平均値を取り出す
        if data.dtype == 'float32':  # 浮動小数点数のときは [0, 1] の値なので普通に足して2で割る
            dataNew = (data[:nframesNew, 0] + data[:nframesNew, 1])/2
        else: # 整数型のときはオーバーフローする可能性があるので，いったん64bit整数にしてから
            data64 = (data[:nframesNew, 0].astype(np.int64) + data[:nframesNew, 1].astype(np.int64))//2
            dataNew = data64.astype(data.dtype)
    
    return framerate, dataNew

WAVE ファイルを入手します．

In [ ]:
# WAVE ファイルを入手
#
### こちらのサイトの素材を利用させてもらってます http://www.ne.jp/asahi/music/myuu/wave/wave.htm

! wget -nc http://www.ne.jp/asahi/music/myuu/wave/cat1.wav
fnCat = 'cat1.wav'

import os

if not os.path.exists(fnCat):
    print(f'{fnCat}のダウンロードがうまくできていないようです')

次のセルを実行すると，`cat1` と `cat2` という二つの配列を作ります．

In [ ]:
# cat1
fr_cat1, cat1 = readWAVE('cat1.wav')
N_cat1 = len(cat1)
print(fr_cat1, N_cat1)

# cat2
cat2 = np.copy(cat1)
N_cat2 = len(cat2)
fr_cat2 = fr_cat1//2
print(fr_cat2, N_cat2)
write('cat2.wav', fr_cat2, cat2.astype('int16'))

`WavFileWarning` という警告が出るかもしれませんが，気にしないでokです．それぞれのWAVEファイルをダウンロードして音を聞いてみてください．

In [ ]:
from google.colab import files

if 1 == 0: # ← の 0 を 1 に修正
    files.download('cat1.wav')
    files.download('cat2.wav')

#### 問題6

上記の `cat1` と `cat2` のデータに DFT を適用して振幅スペクトルを描くと，下図のようになりました．どちらが `cat1` でどちらが `cat2` ですか．


<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/ex13catspec.png">